<a href="https://colab.research.google.com/github/feknall/machinelearning-playground/blob/main/DeepLearning_Assignment_2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install keras-tuner --upgrade

In [ ]:
%load_ext tensorboard

# Preprocessing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist

(train_X, train_y), (test_X, test_y) = mnist.load_data()

print('Train: X=%s, y=%s' % (train_X.shape, train_y.shape))
print('Test: X=%s, y=%s' % (test_X.shape, test_y.shape))

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
plt.hist(train_y, edgecolor='black', bins=np.arange(-0.5, 10))
plt.hist(test_y, edgecolor='black', bins=np.arange(-0.5, 10))
plt.subplots_adjust(top=1.4)
plt.legend(["training", "test"], loc ="top right")

plt.title('Distribution of Digits on Training and Test Dataset')
plt.xlabel('Digit')
plt.ylabel('Count')
plt.xticks(np.arange(0, 10))
plt.show()

In [ ]:
import shutil
import random

import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201

import cv2

import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(train_y[5])

In [ ]:
rows = 5
columns = 10
for digit in range(columns):
		selected = [index for index, value in enumerate(train_y) if value == digit][:rows]
		for index, value in enumerate(selected):
			k = index * columns + digit + 1
			plt.subplot(rows, columns, k)
			plt.imshow(train_X[value], cmap=plt.get_cmap('gray'))
			plt.axis('off')
			plt.subplots_adjust(wspace=0.1, hspace=0.01)
plt.show()

In [ ]:
train_X = train_X.reshape((-1, 28, 28, 1))
test_X = test_X.reshape((-1, 28, 28, 1))

In [ ]:
from tensorflow.keras.utils import to_categorical

train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [ ]:
# convert from integers to floats
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

# normalize to range 0-1
train_X = train_X / 255.0
test_X = test_X / 255.0

print(train_X.shape)
print(train_y.shape)

print(test_X.shape)
print(test_y.shape)

# Base Model


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape,MaxPooling2D

model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
from keras.callbacks import LearningRateScheduler

experiment_train_size = 20000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

history = model.fit(X, y, validation_split=0.2, epochs=30, callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/ai-gras-1/base-model/tensorboard")])

## Reading Tensorboard Log Files

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf

def plot_tensorflow_log(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        # 'compressedHistograms': 100,
        # 'images': 0,
        # 'scalars': 10000,
        # 'histograms': 1000
        'tensors': 200
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    print(event_acc.Tags())

    training_accuracies = event_acc.Tensors('epoch_accuracy')
    tensor_np = tf.make_ndarray(training_accuracies[-1].tensor_proto)

    print(tensor_np)

    y = list()
    x = list()
    for i in range(len(training_accuracies)):
        y.append(tf.make_ndarray(training_accuracies[i].tensor_proto))
        x.append(i)

    return x, y

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf

def plot_tensorflow_log_loss(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        # 'compressedHistograms': 100,
        # 'images': 0,
        # 'scalars': 10000,
        # 'histograms': 1000
        'tensors': 200
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    print(event_acc.Tags())

    training_accuracies = event_acc.Tensors('epoch_loss')
    tensor_np = tf.make_ndarray(training_accuracies[-1].tensor_proto)

    # print(tensor_np)

    y = list()
    x = list()
    for i in range(len(training_accuracies)):
        y.append(tf.make_ndarray(training_accuracies[i].tensor_proto))
        x.append(i)

    return x, y

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf

def plot_tensorflow_log_precision(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        # 'compressedHistograms': 100,
        # 'images': 0,
        # 'scalars': 10000,
        # 'histograms': 1000
        'tensors': 200
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    print(event_acc.Tags())

    training_accuracies = event_acc.Tensors('epoch_precision')
    tensor_np = tf.make_ndarray(training_accuracies[-1].tensor_proto)

    # print(tensor_np)

    y = list()
    x = list()
    for i in range(len(training_accuracies)):
        y.append(tf.make_ndarray(training_accuracies[i].tensor_proto))
        x.append(i)

    return x, y

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf

def plot_tensorflow_log_recall(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        # 'compressedHistograms': 100,
        # 'images': 0,
        # 'scalars': 10000,
        # 'histograms': 1000
        'tensors': 200
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    print(event_acc.Tags())

    training_accuracies = event_acc.Tensors('epoch_recall')
    tensor_np = tf.make_ndarray(training_accuracies[-1].tensor_proto)

    # print(tensor_np)

    y = list()
    x = list()
    for i in range(len(training_accuracies)):
        y.append(tf.make_ndarray(training_accuracies[i].tensor_proto))
        x.append(i)

    return x, y

# Tunning

## Number of Convolution Layers Tunning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D

def build_model(hp):
  conv_layers_count = hp.Int("conv_layers_count", min_value=1, max_value=3, step=1)

  model = Sequential()
  if conv_layers_count == 1:
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
  if conv_layers_count == 2:
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
  if conv_layers_count == 3:
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy", "Precision", "Recall"])
  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=False,
    directory="/content/drive/MyDrive/deep-learning/cnn/",
    project_name="conv_layers_count"
)

In [ ]:
experiment_train_size = 10000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback")],
)

In [ ]:
tuner.results_summary(num_trials=3)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/cnn/conv_layers_count

In [ ]:
!kill 1652

In [ ]:
tuner.search_space_summary(extended=True)

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/cnn/conv_layers_count

In [ ]:
import tensorboard as tb


In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0001/execution0/train/events.out.tfevents.1649367850.979f1a549ef4.72.24.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='1 conv layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0000/execution0/train/events.out.tfevents.1649367808.979f1a549ef4.72.21.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='2 conv layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0002/execution0/train/events.out.tfevents.1649367890.979f1a549ef4.72.27.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='3 conv layer - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0001/execution0/validation/events.out.tfevents.1649367853.979f1a549ef4.72.25.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='1 conv layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0000/execution0/validation/events.out.tfevents.1649367812.979f1a549ef4.72.22.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='2 conv layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0002/execution0/validation/events.out.tfevents.1649367893.979f1a549ef4.72.28.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='3 conv layer - validation')


plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log_loss

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0001/execution0/train/events.out.tfevents.1649367850.979f1a549ef4.72.24.v2"
x, y = function1(log_file)
plt.plot(x, y, label='1 conv layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0000/execution0/train/events.out.tfevents.1649367808.979f1a549ef4.72.21.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2 conv layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0002/execution0/train/events.out.tfevents.1649367890.979f1a549ef4.72.27.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3 conv layer - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0001/execution0/validation/events.out.tfevents.1649367853.979f1a549ef4.72.25.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='1 conv layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0000/execution0/validation/events.out.tfevents.1649367812.979f1a549ef4.72.22.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2 conv layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0002/execution0/validation/events.out.tfevents.1649367893.979f1a549ef4.72.28.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3 conv layer - validation')


plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log_precision

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0001/execution0/train/events.out.tfevents.1649367850.979f1a549ef4.72.24.v2"
x, y = function1(log_file)
plt.plot(x, y, label='1 conv layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0000/execution0/train/events.out.tfevents.1649367808.979f1a549ef4.72.21.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2 conv layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0002/execution0/train/events.out.tfevents.1649367890.979f1a549ef4.72.27.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3 conv layer - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0001/execution0/validation/events.out.tfevents.1649367853.979f1a549ef4.72.25.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='1 conv layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0000/execution0/validation/events.out.tfevents.1649367812.979f1a549ef4.72.22.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2 conv layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0002/execution0/validation/events.out.tfevents.1649367893.979f1a549ef4.72.28.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3 conv layer - validation')


plt.xlabel("Steps")
plt.ylabel("Precision")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log_recall

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0001/execution0/train/events.out.tfevents.1649367850.979f1a549ef4.72.24.v2"
x, y = function1(log_file)
plt.plot(x, y, label='1 conv layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0000/execution0/train/events.out.tfevents.1649367808.979f1a549ef4.72.21.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2 conv layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0002/execution0/train/events.out.tfevents.1649367890.979f1a549ef4.72.27.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3 conv layer - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0001/execution0/validation/events.out.tfevents.1649367853.979f1a549ef4.72.25.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='1 conv layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0000/execution0/validation/events.out.tfevents.1649367812.979f1a549ef4.72.22.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2 conv layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/conv_layers_count/callback/0002/execution0/validation/events.out.tfevents.1649367893.979f1a549ef4.72.28.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3 conv layer - validation')


plt.xlabel("Steps")
plt.ylabel("Recall")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

## Number of Hidden Layers Tunning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D

def build_model(hp):
  hidden_layers_count = hp.Int("hidden_layers_count", min_value=1, max_value=3, step=1)
  
  model = Sequential()

  model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  for i in range(hidden_layers_count):
      model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy", "Precision", "Recall"])

  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/cnn/",
    project_name="dens_layers_count"
)

In [ ]:
experiment_train_size = 10000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback")],
)

In [ ]:
tuner.results_summary(num_trials=3)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/cnn/dens_layers_count

In [ ]:
!kill 1652

In [ ]:
tuner.search_space_summary(extended=True)

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback

In [ ]:
import tensorboard as tb

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0000/execution0/train/events.out.tfevents.1649369360.979f1a549ef4.72.48.v2"
x, y = function1(log_file)
plt.plot(x, y, label='1 dense layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0001/execution0/train/events.out.tfevents.1649369444.979f1a549ef4.72.51.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2 dense layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0002/execution0/train/events.out.tfevents.1649369853.979f1a549ef4.72.54.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3 dense layer - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0000/execution0/validation/events.out.tfevents.1649369364.979f1a549ef4.72.49.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='1 dense layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0001/execution0/validation/events.out.tfevents.1649369447.979f1a549ef4.72.52.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2 dense layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0002/execution0/validation/events.out.tfevents.1649369856.979f1a549ef4.72.55.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3 dense layer - validation')


plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log_loss

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0000/execution0/train/events.out.tfevents.1649369360.979f1a549ef4.72.48.v2"
x, y = function1(log_file)
plt.plot(x, y, label='1 dense layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0001/execution0/train/events.out.tfevents.1649369444.979f1a549ef4.72.51.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2 dense layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0002/execution0/train/events.out.tfevents.1649369853.979f1a549ef4.72.54.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3 dense layer - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0000/execution0/validation/events.out.tfevents.1649369364.979f1a549ef4.72.49.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='1 dense layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0001/execution0/validation/events.out.tfevents.1649369447.979f1a549ef4.72.52.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2 dense layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0002/execution0/validation/events.out.tfevents.1649369856.979f1a549ef4.72.55.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3 dense layer - validation')


plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log_precision

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0000/execution0/train/events.out.tfevents.1649369360.979f1a549ef4.72.48.v2"
x, y = function1(log_file)
plt.plot(x, y, label='1 dense layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0001/execution0/train/events.out.tfevents.1649369444.979f1a549ef4.72.51.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2 dense layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0002/execution0/train/events.out.tfevents.1649369853.979f1a549ef4.72.54.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3 dense layer - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0000/execution0/validation/events.out.tfevents.1649369364.979f1a549ef4.72.49.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='1 dense layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0001/execution0/validation/events.out.tfevents.1649369447.979f1a549ef4.72.52.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2 dense layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0002/execution0/validation/events.out.tfevents.1649369856.979f1a549ef4.72.55.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3 dense layer - validation')


plt.xlabel("Steps")
plt.ylabel("Precision")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log_recall

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0000/execution0/train/events.out.tfevents.1649369360.979f1a549ef4.72.48.v2"
x, y = function1(log_file)
plt.plot(x, y, label='1 dense layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0001/execution0/train/events.out.tfevents.1649369444.979f1a549ef4.72.51.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2 dense layer - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0002/execution0/train/events.out.tfevents.1649369853.979f1a549ef4.72.54.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3 dense layer - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0000/execution0/validation/events.out.tfevents.1649369364.979f1a549ef4.72.49.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='1 dense layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0001/execution0/validation/events.out.tfevents.1649369447.979f1a549ef4.72.52.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2 dense layer - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/dens_layers_count/callback/0002/execution0/validation/events.out.tfevents.1649369856.979f1a549ef4.72.55.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3 dense layer - validation')


plt.xlabel("Steps")
plt.ylabel("Recall")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

## Kernel Size Tunning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D

def build_model(hp):
  kernel_size = hp.Int("kernel_size", min_value=2, max_value=4, step=1)
  
  model = Sequential()

  model.add(Conv2D(16, (kernel_size, kernel_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(32, (kernel_size, kernel_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (kernel_size, kernel_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy", "Precision", "Recall"])

  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/cnn",
    project_name="kernel_size"
)

In [ ]:
experiment_train_size = 10000
X = train_X[:experiment_train_size]
y = train_y[:experiment_train_size]

In [ ]:
tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback")],
)

In [ ]:
tuner.results_summary(num_trials=10)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/cnn/kernel_size

In [ ]:
! kill 5631

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/cnn/kernel_size/callback

In [ ]:
import tensorboard as tb

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0000/execution0/train/events.out.tfevents.1649370340.979f1a549ef4.72.57.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2x2 kernel - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0002/execution0/train/events.out.tfevents.1649370446.979f1a549ef4.72.63.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3x3 kernel - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0001/execution0/train/events.out.tfevents.1649370392.979f1a549ef4.72.60.v2"
x, y = function1(log_file)
plt.plot(x, y, label='4x4 kernel - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0000/execution0/validation/events.out.tfevents.1649370343.979f1a549ef4.72.58.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2x2 kernel - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0002/execution0/validation/events.out.tfevents.1649370449.979f1a549ef4.72.64.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3x3 kernel - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0001/execution0/validation/events.out.tfevents.1649370395.979f1a549ef4.72.61.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='4x4 kernel - validation')


plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log_loss

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0000/execution0/train/events.out.tfevents.1649370340.979f1a549ef4.72.57.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2x2 kernel - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0002/execution0/train/events.out.tfevents.1649370446.979f1a549ef4.72.63.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3x3 kernel - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0001/execution0/train/events.out.tfevents.1649370392.979f1a549ef4.72.60.v2"
x, y = function1(log_file)
plt.plot(x, y, label='4x4 kernel - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0000/execution0/validation/events.out.tfevents.1649370343.979f1a549ef4.72.58.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2x2 kernel - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0002/execution0/validation/events.out.tfevents.1649370449.979f1a549ef4.72.64.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3x3 kernel - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0001/execution0/validation/events.out.tfevents.1649370395.979f1a549ef4.72.61.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='4x4 kernel - validation')


plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log_precision

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0000/execution0/train/events.out.tfevents.1649370340.979f1a549ef4.72.57.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2x2 kernel - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0002/execution0/train/events.out.tfevents.1649370446.979f1a549ef4.72.63.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3x3 kernel - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0001/execution0/train/events.out.tfevents.1649370392.979f1a549ef4.72.60.v2"
x, y = function1(log_file)
plt.plot(x, y, label='4x4 kernel - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0000/execution0/validation/events.out.tfevents.1649370343.979f1a549ef4.72.58.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2x2 kernel - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0002/execution0/validation/events.out.tfevents.1649370449.979f1a549ef4.72.64.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3x3 kernel - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0001/execution0/validation/events.out.tfevents.1649370395.979f1a549ef4.72.61.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='4x4 kernel - validation')


plt.xlabel("Steps")
plt.ylabel("Precision")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

function1 = plot_tensorflow_log_recall

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0000/execution0/train/events.out.tfevents.1649370340.979f1a549ef4.72.57.v2"
x, y = function1(log_file)
plt.plot(x, y, label='2x2 kernel - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0002/execution0/train/events.out.tfevents.1649370446.979f1a549ef4.72.63.v2"
x, y = function1(log_file)
plt.plot(x, y, label='3x3 kernel - train')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0001/execution0/train/events.out.tfevents.1649370392.979f1a549ef4.72.60.v2"
x, y = function1(log_file)
plt.plot(x, y, label='4x4 kernel - train')


log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0000/execution0/validation/events.out.tfevents.1649370343.979f1a549ef4.72.58.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='2x2 kernel - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0002/execution0/validation/events.out.tfevents.1649370449.979f1a549ef4.72.64.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='3x3 kernel - validation')

log_file = "/content/drive/MyDrive/deep-learning/cnn/kernel_size/callback/0001/execution0/validation/events.out.tfevents.1649370395.979f1a549ef4.72.61.v2"
x, y = function1(log_file)
plt.plot(x, y, '--', label='4x4 kernel - validation')


plt.xlabel("Steps")
plt.ylabel("Recall")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

In [ ]:
best_models[0].get_config()

In [ ]:
best_models[0].summary()

In [ ]:
import visualkeras
from PIL import ImageFont

font = ImageFont.truetype("/content/font/Roboto-Regular.ttf", 12)
visualkeras.layered_view(best_models[0], to_file='output.png', legend=True, font=font).show()

In [ ]:
best_models[0].summary()

# System Spec

In [ ]:
! python --version

In [ ]:
! python -c 'import keras; print(keras.__version__)'

In [ ]:
! python -c 'import tensorflow as tf; print(tf.__version__)'

In [ ]:
! nvidia-smi -L

In [ ]:
! cat /proc/cpuinfo

# Installation

## Visual Keras

In [ ]:
! pip install git+https://github.com/paulgavrikov/visualkeras

# Visualization

In [ ]:
model = Sequential()
kernel_size=3
model.add(Conv2D(16, (kernel_size, kernel_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (kernel_size, kernel_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (kernel_size, kernel_size), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))

model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.build((None, 28, 28, 1))
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
import visualkeras
from PIL import ImageFont

font = ImageFont.truetype("/content/Roboto-Regular.ttf", 12)

visualkeras.layered_view(model, to_file='output.png', legend=True, font=font).show()